In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('2015riversdatacleaned.xlsx', sheet_name = 'Sheet2')

In [3]:
df['temp_avg'] = (df['min_temp'] + df['max_temp'])/2
df['do_avg'] = (df['dissolved_oxygen_min'] + df['dissolved_oxygen_max'])/2
df['ph_avg'] = (df['ph_min'] + df['ph_max'])/2
df['conductivity_avg'] = (df['conductivity_min'] + df['conductivity_max'])/2
df['bod_avg'] = (df['bod_min'] + df['bod_max'])/2
df['fc_avg'] = (df['fc_min'] + df['fc_max'])/2

In [4]:
def calculate_do_index(temp, do):
    '''this function returns the DO index for the input value of average temperature, and average do'''
    docp = ((np.exp(7.7117-1.31403*np.log10(temp+45.93))) * 1 * (1-np.exp(11.8571-(3840.7/(temp+273.15))-(216961/((temp+273.15)**2)))/1)* (1-(0.000975-(0.00001426*temp)+(0.00000006436*(temp**2)))*1))/ (1-np.exp(11.8571-(3840.7/(temp+273.15))-(216961/((temp+273.15)**2))))/ (1-(0.000975-(0.00001426*temp)+(0.00000006436*(temp**2))))
    dosp = 100*do/docp
    if dosp <= 40:
        return dosp * 0.66 + 0.18
    elif (dosp > 40) & (dosp <= 100):
        return -13.55+1.17 * dosp
    else:
        return 163.34-0.62*dosp

In [5]:
def calculate_bod_index(value):
    '''this function returns the value of BOD index for the input value of average BOD'''
    if value <= 10:
        return 96.67 - 7 * value
    elif (value > 10) & (value <= 30):
        return 38.9 - 1.23 * value
    else:
        return 2

In [6]:
def calculate_ph_index(value):
    '''this function returns the value of pH index for the input value of average pH'''
    if value <= 2:
        return 0
    elif (value > 2) & (value <= 5):
        return 16.1 + 7.35*value
    elif (value > 5) & (value <= 7.3):
        return -142.67 + 33.5*value
    elif (value > 7.3) & (value <= 10):
        return 316.96 - 29.85*value
    elif (value <= 12) & (value > 10):
        return 96.17 - 8*value
    else:
        return 0

In [7]:
def calculate_fc_index(value):
    '''this function returns the value of FC index for the input value of average faecal coliform'''
    if (value > 1) & (value <= 172):
        return 97.2 - 26.6*np.log10(value)
    elif (value > 1000) & (value <= 100000):
        return 42.33-7.75*np.log10(value)
    else:
        return 2

In [8]:
def calculate_wqi(do, bod, ph, fc):
    '''this function returns wqi for the input values of do, bod, ph, and fc'''
    return do*0.31 + bod*0.19 + ph*0.22 + fc*0.28

In [9]:
def check_drinking_safe(do, bod, ph, fc):
    '''this function takes the value of ph, do, bod, and tc and returns 1 if the water is safe'''
    if (do >= 6) & (bod <= 2) & (fc <= 50) & (ph >= 6.5) & (ph <= 8.5):
        return 1
    else:
        return 0

In [10]:
def check_bathing_safe(do, bod, ph, fc):
    '''this function takes the value of ph, do, bod, and tc and returns 1 if the water is safe'''
    if (do >= 5) & (bod <= 3) & (fc <= 500) & (ph <=8.5) & (ph >= 6.5):
        return 1
    else:
        return 0

In [11]:
def check_industrial_safe(ph, ec):
    '''this function takes the value of ph, and ec returns 1 if the water is safe'''
    if (ph >= 6) & (ph <= 8.5) & (ec <= 2250):
        return 1
    else:
        return 0

In [12]:
df['ph_index'] = df.ph_avg.apply(calculate_ph_index)
df['fc_index'] = df.fc_avg.apply(calculate_fc_index)
df['bod_index'] = df.bod_avg.apply(calculate_bod_index)
df['do_index'] = df.apply(lambda x: calculate_do_index(x.temp_avg, x.do_avg), axis=1)

In [13]:
df['drinking_safe'] = df.apply(lambda x: check_drinking_safe(x.do_avg, x.bod_avg, x.ph_avg, x.fc_avg), axis = 1)
df['bathing_safe'] = df.apply(lambda x: check_bathing_safe(x.do_avg, x.bod_avg, x.ph_avg, x.fc_avg), axis = 1)
df['industrial_safe'] = df.apply(lambda x: check_industrial_safe(x.ph_avg, x.conductivity_avg), axis = 1)

In [14]:
df['wqi'] = df.apply(lambda x: calculate_wqi(x.do_index, x.bod_index, x.ph_index, x.fc_index), axis = 1)

In [15]:
df.to_csv('2015_indices.csv', index=None)

In [16]:
df_loc = pd.read_csv('CPCB_Locations_Coordinates.csv')

In [17]:
df_loc.sample(10)

,id,river,river_code,state_name,state_code,location,location_clean,location_wo_comma,location_code,gpslat,gpslong
84,154,Ganga,NaN,West Bengal,NaN,Ganga At Dakshineshwar,"dakshineshwar, kolkata",dakshineshwar,1053,22.65779,88.34890
578,1813,Thirumanimuthar,NaN,Madhya Pradesh,NaN,Kunda At Khargone,khargone,khargone,2114,21.82437,75.57793
20,744,Pennar,NaN,Andhra Pradesh,NaN,"Pennar Siddvata, Nellore",siddavatam,siddavatam,30,14.46789,78.95939
1063,300,Kshipra,NaN,Madhya Pradesh,NaN,"Kshipra At U/S Of Mahidpur City,_x005F_x005F_x...","mahidpur city, ujjain",mahidpur city,3316,23.48906,75.62099
972,268,Kali Sind,NaN,Rajasthan,NaN,River Kali Sindh At Barod_x005F_x005F_x000D_Ro...,"barod road bridge, kota",barod road bridge,2956,25.14094,75.79205
1101,1748,Thirumanimuthar,NaN,Meghalaya,NaN,"Wah Blei At Shdaddkhar, West Khasi Hills","shadaddkhar, west khasi hills",shadaddkhar,3374,19.06844,72.83491
1078,1038,Umkhen,NaN,Meghalaya,NaN,Umkhen At Wahkdait,wahkdait,wahkdait,3350,25.20885,91.97682
755,186,Varuna,NaN,Uttar Pradesh,NaN,"Varuna At Rameshwar,_x005F_x005F_x000D_Varanasi","rameshwar, varanasi",rameshwar,2482,25.38570,82.85150
1042,68,Jhelum,NaN,Jammu & _x005F_x005F_x000D_Kashmir,NaN,Jhelum At Verinag,"verinag, jammu and kashmir",verinag,3272,28.63374,77.20601
6,455,Tapi,NaN,Madhya Pradesh,NaN,Tapi At Burhanpur,burhanpur,burhanpur,10,21.31765,76.18291


In [20]:
df.sample(10)

,location_code,river,state,min_temp,max_temp,dissolved_oxygen_min,dissolved_oxygen_max,ph_min,ph_max,conductivity_min,...,bod_avg,fc_avg,ph_index,fc_index,bod_index,do_index,drinking_safe,bathing_safe,industrial_safe,wqi
266,2950.0,"DHANARI DAM, NEAR SWAROOPGANJ, SIROHI, RAJASTHAN",RAJASTHAN,19.5,27.0,5.1,6.5,7.5,9.0,170.0,...,1.35,5.0,70.6975,78.607398,87.22,2.102454,0,1,1,54.787082
609,1181.0,"KRISHNA AT D/S OF NARAYANPURA DAM, KARNATAKA",KARNATAKA,28.0,41.0,5.0,11.6,7.3,9.3,380.0,...,1.65,700.0,69.2050,2.000000,85.12,3.178124,0,0,1,32.943118
507,1042.0,BRAHMANI AT KAMALANGA,ODISHA,24.0,38.0,6.6,9.6,7.3,8.4,144.0,...,2.10,12245.0,82.6375,10.648319,81.97,3.032527,0,0,1,37.676163
444,1101.0,"MAHANADI AT INTERSTATE BOURY, CHHATISGARH.",CHHATTISGARH,19.0,26.0,6.8,7.3,7.3,7.5,190.0,...,1.55,NaN,96.0700,2.000000,85.82,2.502284,0,0,1,38.776908
194,3127.0,"SONE RIVER AT BAHIYARA, BIHAR",BIHAR,23.0,34.0,7.6,9.0,7.5,9.0,206.0,...,1.60,500.0,70.6975,2.000000,85.47,3.048368,0,1,1,33.297744
483,1279.0,KUAKHAI U/S AT BHUBANESWAR,ODISHA,22.0,32.0,5.7,9.0,7.5,8.4,183.0,...,1.30,11650.0,79.6525,10.815974,87.57,2.690720,0,0,1,38.024446
699,1911.0,CHANDRABHAGA U/S OF PAHARPUR TOWN,MAHARASHTRA,26.0,39.0,4.4,7.5,7.6,8.7,633.0,...,7.55,275.0,73.6825,2.000000,43.82,2.298594,0,0,1,25.808514
17,1021.0,"SATLUJ AT BOAT BDG. DHARMKOTNAKODAR ROAD, JALA...",PUNJAB,15.0,26.0,5.5,8.0,7.2,7.7,351.0,...,7.00,7140.0,94.5775,12.463839,47.67,2.366142,0,0,1,34.087729
769,1201.0,"BHAVANI AT\nPATHIRAKALIAMMAN KOIL, TAMILNADU",TAMIL NADU,26.0,28.0,4.6,8.5,6.6,8.3,75.0,...,0.90,1089.0,94.5775,18.793034,90.37,2.417445,0,0,1,43.988807
106,2486.0,"GANGA D/S, MIRZAPUR",UTTAR PRADESH,19.0,28.0,8.1,8.7,7.8,7.9,366.0,...,2.40,1700.0,82.6375,17.294021,79.87,2.969981,0,0,1,39.118570


In [19]:
df.rename(columns={'station_code':'location_code'}, inplace = True)

In [21]:
ndf = df.merge(df_loc, how='left', on = 'location_code')

In [22]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 784 entries, 0 to 783
Data columns (total 43 columns):
location_code           783 non-null float64
river_x                 783 non-null object
state                   783 non-null object
min_temp                757 non-null float64
max_temp                757 non-null float64
dissolved_oxygen_min    768 non-null float64
dissolved_oxygen_max    768 non-null float64
ph_min                  781 non-null float64
ph_max                  781 non-null float64
conductivity_min        743 non-null float64
conductivity_max        742 non-null float64
bod_min                 775 non-null float64
bod_max                 775 non-null float64
nitrate_min             741 non-null float64
nitrate_max             741 non-null float64
fc_min                  606 non-null float64
fc_max                  606 non-null float64
tc_min                  641 non-null float64
tc_max                  641 non-null float64
temp_avg                757 non-null float

In [24]:
ndf.to_csv('2015_indices_with_location.csv', index = None)